In [ ]:
import pandas as pd
import numpy as np
import os
import subprocess

In [ ]:
drop_cols_aseg_vol = [
    "BrainSegVolNotVent",
    "BrainSegVolNotVentSurf",
    "eTIV",
    "MeanThickness",
    "EstimatedTotalIntraCranialVol",
    "MaskVol",
    "BrainSegVol-to-eTIV",
    "MaskVol-to-eTIV",
    "SurfaceHoles",
] 

In [ ]:
#load all tsvs from /opt/notebooks/freesurfer_stats
data_dir = "../00_data/"
for source_folder in os.listdir(data_dir):
    dfs_dict={}
    source_folder = data_dir + source_folder + "/"
    for file in os.listdir(source_folder):
        if file.endswith('.csv'):
            if file.startswith("lh"):
                lh_df = pd.read_csv(f'{source_folder}'+file)
                rh_df = pd.read_csv(f'{source_folder}rh'+file[2:])
                lh_df.rename(columns={lh_df.columns[0]: 'ID'}, inplace=True)
                rh_df.rename(columns={rh_df.columns[0]: 'ID'}, inplace=True)
                df = pd.merge(lh_df, rh_df, on='ID')
                df["ID"] = df["ID"].str.replace("sub-", "",)
                name = file[3:].replace(".csv", "")
            elif file.startswith("rh") == False:
                df = pd.read_csv(f'{source_folder}'+file)
                df.rename(columns={df.columns[0]: 'ID'}, inplace=True)
                df["ID"] = df["ID"].str.replace("sub-", "")
                name = file.replace(".csv", "")
            else:
                continue
            dfs_dict[name] = df
    for key in dfs_dict.keys():
        print(key)
    keys = ['aseg.volume', 'aparc.volume', 'aparc.thickness']
    for key in list(dfs_dict.keys()):  
        if key not in keys: 
            del dfs_dict[key]
    for key, df in dfs_dict.items():
        print(key)
        for col in df.columns:
            print(df[col].describe())
    for key, df in dfs_dict.items():
        drop_cols = [col for col in df.columns if any(drop_col in col for drop_col in drop_cols_aseg_vol)]
        df.drop(columns=drop_cols, inplace=True)
        dfs_dict[key] = df
        print(len(df))
        print(len(df.columns))

    # Create the directory if it does not exist
    output_dir = os.path.join(source_folder, "freesurfer_finished")
    os.makedirs(output_dir, exist_ok=True)

    # Save individual dataframes
    for key, df in dfs_dict.items():
        print(len(df.columns))
        df.to_csv(os.path.join(output_dir, f'{key}.csv'), index=False)

In [ ]:
#command = "dx upload -r /opt/notebooks/freesurfer_finished/*.csv --path project-GvJ4qQ8Jk6k7QFZ0Jzjp7vJ6:/Users/Esra_Lenz/02_Data/00_mri_tables/00_important_freesurfer_stats/"
#subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, universal_newlines=True)